# Code Search on Kubeflow

This notebook implements an end-to-end Semantic Code Search on top of [Kubeflow](https://www.kubeflow.org/) - given an input query string, get a list of code snippets semantically similar to the query string.

**NOTE**: For instructions on how to get this notebook, see [kubeflow/examples/code_search](https://github.com/kubeflow/examples/tree/master/code_search).

## Install dependencies

Let us install all the Python dependencies. Note that everything must be done with `Python 2`. This will take a while and only needs to be run once.

In [ ]:
! pip2 install -r src/requirements.txt

# FIXME(sanyamkapoor): The Kubeflow Batch Prediction dependency is installed from a fork for reasons in
# kubeflow/batch-predict#9 and corresponding issue kubeflow/batch-predict#10
! pip2 install https://github.com/activatedgeek/batch-predict/tarball/fix-value-provider

# Only for BigQuery cells
! pip2 install google-cloud-bigquery[pandas]

In [ ]:
%load_ext google.cloud.bigquery

### Configure Variables

This involves setting up the Ksonnet application as well as utility environment variables for various CLI steps.

In [ ]:
# Configuration Variables. Modify as desired.

PROJECT = 'kubeflow-dev'
CLUSTER_NAME = 'kubeflow-latest'
CLUSTER_REGION = 'us-east1-d'
CLUSTER_NAMESPACE = 'kubeflow-latest'

TARGET_DATASET = 'code_search'
WORKING_DIR = 'gs://kubeflow-examples/t2t-code-search/20180809'
WORKER_MACHINE_TYPE = 'n1-highcpu-32'
NUM_WORKERS = 16

# DO NOT MODIFY. These are environment variables to be used in a
# bash shell.
%env PROJECT $PROJECT
%env CLUSTER_NAME $CLUSTER_NAME
%env CLUSTER_REGION $CLUSTER_REGION
%env CLUSTER_NAMESPACE $CLUSTER_NAMESPACE

%env TARGET_DATASET $TARGET_DATASET
%env WORKING_DIR $WORKING_DIR
%env WORKER_MACHINE_TYPE $WORKER_MACHINE_TYPE
%env NUM_WORKERS $NUM_WORKERS

###  Setup Authorization

In a Kubeflow cluster, we already have the key credentials available with each pod and will re-use them to authenticate. This will allow us to submit `TFJob`s and execute `Dataflow` pipelines.

In [ ]:
%%bash

# FIXME: The deployment must set up the service account with GKE permissions
gcloud auth activate-service-account --key-file=${GOOGLE_APPLICATION_CREDENTIALS}
gcloud container clusters get-credentials ${CLUSTER_NAME} --region ${CLUSTER_REGION}
kubectl config set contexts.$(kubectl config current-context).namespace ${CLUSTER_NAMESPACE}

### Verify Version Information

In [ ]:
%%bash

echo "Pip Version Info: " && pip2 --version && echo
echo "Google Cloud SDK Info: " && gcloud --version && echo
echo "Ksonnet Version Info: " && ks version && echo
echo "Kubectl Version Info: " && kubectl version

## Pre-Processing Github Files

In this step, we will run a [Google Cloud Dataflow](https://cloud.google.com/dataflow/) pipeline (based on Apache Beam). A `Python 2` module `code_search.dataflow.cli.preprocess_github_dataset` has been provided which builds an Apache Beam pipeline. A list of all possible arguments can be seen via the following command.

In [ ]:
%%bash

cd src

python2 -m code_search.dataflow.cli.preprocess_github_dataset -h

### Configuration

We use a subset of the options available to run our Dataflow job. The variables are required and make sure you modify the preset variables as desired.

### Run the Dataflow Job for Pre-Processing

In [ ]:
%%bash

# See help for a short description of each argument.

python2 -m code_search.dataflow.cli.preprocess_github_dataset \
        --runner DataflowRunner \
        --project "${PROJECT}" \
        --target_dataset "${TARGET_DATASET}" \
        --data_dir "${WORKING_DIR}/data" \
        --job_name "preprocess-github-dataset" \
        --temp_location "${WORKING_DIR}/data/dataflow/temp" \
        --staging_location "${WORKING_DIR}/data/dataflow/staging" \
        --worker_machine_type "${WORKER_MACHINE_TYPE}" \
        --num_workers "${NUM_WORKERS}"

When completed successfully, this should create a dataset in `BigQuery` named `target_dataset`. Additionally, it also dumps CSV files into `data_dir` which contain training samples (pairs of function and docstrings) for our Tensorflow Model.

## Prepare Dataset for Training

In this step we will use `t2t-datagen` to convert the transformed data above into the `TFRecord` format. We will run this job on the Kubeflow cluster.

In [ ]:
%%bash

ks apply code-search -c t2t-code-search-datagen

## Execute Tensorflow Training

In [ ]:
%%bash

ks apply code-search -c t2t-code-search-trainer

## Export Tensorflow Model

In [ ]:
%%bash

ks apply code-search -c t2t-code-search-exporter

## Compute Function Embeddings

In this step, we will use the exported model above to compute function embeddings via another `Dataflow` pipeline. A `Python 2` module `code_search.dataflow.cli.create_function_embeddings` has been provided for this purpose. A list of all possible arguments can be seen below.

In [ ]:
%%bash

python2 -m code_search.dataflow.cli.create_function_embeddings -h

### Configuration

First, select a Exported Model version from the `${WORKING_DIR}/output/export/Servo`. This should be name of a folder with UNIX Seconds Timestamp like `1533685294`.

In [ ]:
MODEL_VERSION = '1533685294'

# DO NOT MODIFY. The following statements are needed to support
# environment variables in a bash cell.
%env MODEL_VERSION $MODEL_VERSION

### Run the Dataflow Job for Function Embeddings

In [ ]:
%%bash

python2 -m code_search.dataflow.cli.create_function_embeddings \
        --runner DataflowRunner
        --project "${PROJECT}" \
        --target_dataset "${TARGET_DATASET}" \
        --problem github_function_docstring \
        --data_dir "${WORKING_DIR}/data" \
        --saved_model_dir "${WORKING_DIR}/output/export/Servo/${MODEL_VERSION}" \
        --job_name compute-function-embeddings
        --temp_location "${WORKING_DIR}/data/dataflow/temp" \
        --staging_location "${WORKING_DIR}/data/dataflow/staging" \
        --worker_machine_type "${WORKER_MACHINE_TYPE}" \
        --num_workers "${NUM_WORKERS}"

When completed successfully, this should create another table in the same `BigQuery` dataset which contains the function embeddings for each existing data sample available from the previous Dataflow Job. Additionally, it also dumps a CSV file containing metadata for each of the function and its embeddings.

## Create Search Index

We now create the Search Index from the computed embeddings so that during a query we can do a k-Nearest Neighbor search to give out semantically similar results.

In [ ]:
%%bash

ks apply code-search -c search-index-creator

Using the CSV files generated from the previous step, this creates an index using [NMSLib](https://github.com/nmslib/nmslib). A unified CSV file containing all the code examples for a human-readable reverse lookup during the query, is also created in the `WORKING_DIR`.

## Deploy an Inference Server

We've seen offline inference during the computation of embeddings. For online inference, we deploy the exported Tensorflow model above using [Tensorflow Serving](https://www.tensorflow.org/serving/).

In [ ]:
%%bash

ks apply code-search -c t2t-code-search-serving

## Deploy Search UI

We finally deploy the Search UI which allows the user to input arbitrary strings and see a list of results corresponding to semantically similar Python functions. This internally uses the inference server we just deployed.

In [ ]:
%%bash

ks apply code-search -c search-index-server

The service should now be available at FQDN of the Kubeflow cluster at path `/code-search/`.